# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [ ]:
!python -m pip install semantic-kernel -U

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion("gpt-35-turbo", endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion("gpt-3.5-turbo-16k", api_key, org_id))

## It all begins with an ask

In [3]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [4]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [7]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [8]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [9]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [10]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [11]:
results = await planner.execute_plan_async(new_plan, kernel)

In [12]:
print(results)

ASSUREZ-VOUS D'UTILISER UNIQUEMENT LE FRANÇAIS.

1. UN FESTIN PITTORESQUE DE FLAMMES VACILLANTES, MON AMOUR,
   AU SEIN DES MURS DE NOTRE HUMBLE DEMEURE.
2. UN PIQUE-NIQUE DANS LE PARC VERDOYANT, CHER CŒUR,
   OÙ LA GÉNÉROSITÉ DE LA NATURE SERA NOTRE BANQUET.
3. UNE JOURNÉE DE REPOS PARTAGÉ AU SPA, BELLE ÂME,
   POUR FAIRE FONDRE NOS SOUCIS ET NOURRIR NOS ESPRITS.
4. UNE DÉGUSTATION DU NECTAR DE LA VIGNE,
   PARMI LES VIGNES DE CE ROYAUME LOCAL.
5. ENSEMBLE, PARCOURONS UN CHEMIN PITTORESQUE,
   ET BUVONS LA BEAUTÉ QUE NOTRE TERRE OFFRE.
6. PRÈS DE LA CHEMINÉE, CRÉONS UN REPAS,
   ENLACÉS DANS L'AMOUR ET LE DÉLICE CULINAIRE.
7. S'ÉMERVEILLER DEVANT UNE GRANDE PERFORMANCE EN DIRECT,
   ET SE PERDRE DANS L'ÉTREINTE DE L'ART.
8. UNE NUIT DE CINÉMA, AVEC DES RÉCITS AMOUREUX,
   QUI ÉVEILLENT NOS CŒURS ET ENFLAMMENT NOS RÊVES.
9. ENSEMBLE, APPRENONS L'ART DE LA DANSE,
   ET GLISSONS SUR LE SOL DANS LA PLUS DOUCE DES RYTHMES.
10. UNE ESCAPADE D'UN WEEK-END DANS UN CHALET DOUILLET,
    OÙ LE F

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [13]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [ ]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [ ]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Let's ask the sequential planner to execute the plan.

In [ ]:
result = await sequential_plan.invoke_async()

In [ ]:
print(result)

# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [2]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [3]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240dbb10>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240dbe90>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240e4250>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240e45d0>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240e4950>}

In [18]:
ask = "What is the sum of 110 and 990?"

In [19]:
plan = await planner.create_plan_async(goal=ask)

In [38]:
result = await plan.invoke_async()

In [39]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [4]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [5]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [6]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240f3c10>}

Let's also add a couple more skills

In [8]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240f35d0>,
 'Subtract': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1240fed90>}

In [10]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [11]:
ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

In [18]:
functions = planner.get_function_descriptions()
functions

"WebSearch.searchAsync: Performs a web search for a given query\n  inputs:\n    - query: The search query\n\nfileIO.readAsync: Read a file\n  inputs:\n    - input: Path of the source file\n\nfileIO.writeAsync: Write a file\n  inputs:\n    - content: File content\n    - path: Destination path\n\nmath.Add: Adds value to a value\n  inputs:\n    - input: The value to add\n    - Amount: Amount to add\n\nmath.Subtract: Subtracts value to a value\n  inputs:\n    - input: The value to subtract\n    - Amount: Amount to subtract\n\ntext.lowercase: Convert a string to lowercase.\n  inputs: None\n\ntext.trim: Trim whitespace from the start and end of a string.\n  inputs: None\n\ntext.trim_end: Trim whitespace from the end of a string.\n  inputs: None\n\ntext.trim_start: Trim whitespace from the start of a string.\n  inputs: None\n\ntext.uppercase: Convert a string to uppercase.\n  inputs: None\n\nthis.EdgeCaseExamples: List a few edge case examples of plans to handle\n  inputs:\n    - goal: The cu

In [19]:
result = await plan.invoke_async()
print(result)

Result not found, review _steps_taken to see what happened.
[
    {
        "thought": "To determine the total number of championships combined for the top 5 teams in the NBA, we need to gather information about the number of championships each team has won. We can then sum up the total number of championships for the top 5 teams.\n\n[QUESTION]\nHow can we gather information about the number of championships each team has won?",
        "action": null,
        "action_variables": {},
        "observation": null,
        "final_answer": null,
        "original_response": "[THOUGHT]\nTo determine the total number of championships combined for the top 5 teams in the NBA, we need to gather information about the number of championships each team has won. We can then sum up the total number of championships for the top 5 teams.\n\n[QUESTION]\nHow can we gather information about the number of championships each team has won?"
    },
    {
        "thought": "We can search for the number of ch

In [50]:
print(result)

The top 5 teams in the NBA have a combined total of 34 championships.


Let's see the steps that the AI took to get to the answer.

In [51]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  To answer this question, we need to determine the top 5 teams in the NBA and find out the total number of championships they have won. We can do this by gathering information about the teams and their championship records.
  [THOUGHT]
  [ACTION]
  {
    "action": "WebSearch.searchAsync",
    "action_variables": {
      "query": "top 5 teams in the NBA"
    }
  }
  [ACTION]
  {"action": "WebSearch.searchAsync", "action_variables": {"query": "top 5 teams in the NBA"}}
  [OBSERVATION]
  ['Teams of the Week Make It Last Forever: Atlanta (4-0) — A little defense can go a long way. Something Just Ain’t Right: Sacramento (0-2) — A new record for the longest playoff drought in NBA...', 'Breaking down stats and film on what Kevin Durant, Kyrie Irving, Russell Westbrook and others have brought 